In [ ]:
! docker run -d -p 11434:11434 --name ollama ollama/ollama:latest

In [ ]:
! docker exec ollama ollama run deepseek-r1:1.5b

In [ ]:
! pip install langchain langchain_community langchain-openai scikit-learn langchain-ollama

In [1]:
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_ollama import OllamaEmbeddings
from langchain_core.documents import Document
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from inspect import signature
from yaml import safe_load
import pandas as pd 
import tqdm
import re
import json

In [34]:
class XRIFGenerator:
    def __init__(self, 
                 prompt_template: str,
                 waypoints_csv: str,
                 chat_model: str = "deepseek-r1:1.5b",
                 embed_model: str = "deepseek-r1:1.5b",
                 chat_kwargs: dict = {"num_predict": 500, "temperature": 0.5, "top_p": 0.5}):
        self.waypoints_list = []
        self.kwargs = chat_kwargs
        self.prompt_template = prompt_template
        self.waypoints_csv = waypoints_csv
        self.x_coord = 0
        self.y_coord = 0
        print(f"Loading model: {chat_model}...")
        
        embed = OllamaEmbeddings(model = embed_model)
        self.vector_store = SKLearnVectorStore(embed)
        self.llm = OllamaLLM(model=chat_model, **self.filter_kwargs_for_OllamaLLM(self.kwargs))
        self.output_parser = StrOutputParser()

        waypoints = pd.read_csv(self.waypoints_csv)
        waypoint_cols = ['Location', 'X co-ordinate', 'Y co-ordinate', 'Floor', 'Section', 'Keywords']
        given_waypoints_cols = list(waypoints.columns)
        if given_waypoints_cols != waypoint_cols:
            raise Exception("Columns do not match, please provide a new dataset")
        
        num_rows = len(waypoints)
        print(f"Processing {num_rows} waypoints...")
        with tqdm.tqdm(total=num_rows) as pbar:
            for index, row in waypoints.iterrows():
                self.waypoints_list.append(Document(id = f"waypoint_{index}", page_content=f"Location Name/Part Number: {row['Location']} X-Coordinate: {row['X co-ordinate']} Y-Coordinate: {row['Y co-ordinate']} Section: {row['Section']} Keywords: {row['Keywords']}"))
                pbar.update(1)
        
        print("Adding waypoints to vector store...")
        self.vector_store.add_documents(self.waypoints_list)
        print("Creating retriever...")
        self.retriever = self.vector_store.as_retriever(
            search_type="similarity_score_threshold",
            search_kwargs={'score_threshold': 0.3}
            )

        with open(self.prompt_template, 'r') as file:
            prompt_dict = safe_load(file)
            self.prompt = prompt_dict['prompt']
            self.prompt_name = prompt_dict['prompt_name']
        
        print("Creating prompt template...")
        self.prompt_template = PromptTemplate(template=self.prompt, input_variables=["documents", "query", "starting_location"])

        print("Creating RAG chain...")
        self.rag_chain = self.prompt_template | self.llm | self.output_parser
        
    def filter_kwargs_for_OllamaLLM(self, kwargs: dict) -> dict:
        chatollama_params = signature(OllamaLLM.__init__).parameters
        return {k: v for k, v in kwargs.items() if k in chatollama_params}
    
    def generate_xrif_with_deepseek_model(self, query: str):
        xrif_response = None
        documents = self.retriever.get_relevant_documents(query)
        doc_texts = "\\n".join([doc.page_content for doc in documents])

        llm_response = self.rag_chain.invoke({"documents": doc_texts, "query": query, "starting_location": f"X-Coordinate: {self.x_coord}, Y-Coordinate: {self.y_coord}"})

        xrif_response = re.sub(r"<think>.*?</think>", "", llm_response, flags=re.DOTALL)

        xrif_response = re.sub(r"\n", "", xrif_response)

        xrif_response = re.sub(r"\\n", "", xrif_response)

        start = xrif_response.find('{')
        end = xrif_response.rfind('}')

        if start != -1 and end != -1:
            xrif_response = xrif_response[start:end+1]
            try:
                xrif_response = json.loads(xrif_response)
            except json.JSONDecodeError as e:
                print("Error parsing JSON response")

        
        return llm_response, xrif_response
    
    def batch_test_run(self, df: pd.DataFrame):
        return_df = df.copy()
        num_rows = len(df)
        print(f"Processing {num_rows} test prompts...")
        with tqdm.tqdm(total=num_rows) as pbar:
            for index, row in df.iterrows():
                error_message = ''
                note = ''
                query = row['Prompt']
                self.x_coord = row['Starting X']
                self.y_coord = row['Starting Y']
                expected_response = row['Expected Response']
                llm_response, xrif_response = self.generate_xrif_with_deepseek_model(query)
                return_df.at[index, 'Full LLM Response'] = str(xrif_response)
                return_df.at[index, 'XRIF Generated'] = json.dumps(xrif_response) if isinstance(xrif_response, dict) else str(xrif_response)
                expected_response = list(expected_response)
                if xrif_response and type(xrif_response) == dict:
                    if 'actions' in xrif_response.keys():
                        actions = xrif_response['actions']
                        if len(expected_response) == len(actions):
                            if row['Expected Response Type'] == 'Nav':
                                list_of_locations = [action['input']['name'] for action in actions if (action['action'] == 'navigate') and ('input' in action) and ('name' in action['input'])]
                                set_of_locations = set(list_of_locations)
                                set_of_expected_locations = set(expected_response)
                                if set_of_locations == set_of_expected_locations:
                                    note = "All locations are present in the response"
                                    return_df.at[index, 'Notes'] = note
                            for i in range(len(xrif_response['actions'])):
                                if actions[i]['action'] == 'navigate':
                                    if 'input' in actions[i]:
                                        if 'name' in actions[i]['input'].keys():
                                            if actions[i]['input']['name'] != expected_response[i]:
                                                error_message = f"Expected location name: {actions[i]['name']} does not match with the provided location name: {expected_response[i]}"
                                                return_df.at[index, 'Error Message'] = error_message
                                                break
                                        else:
                                            error_message = f"Response Error Missing field: 'name' in action object {i}"
                                            return_df.at[index, 'Error Message'] = error_message
                                            break
                                    else:
                                        error_message = f"Response Error Missing field: 'input' in action object {i}"
                                        return_df.at[index, 'Error Message'] = error_message
                                        break
                                elif actions[i]['action'] == 'wait':
                                        if 'input' in actions[i]:
                                                if actions[i]['action'] != expected_response[i][0]:
                                                    error_message = f"Expected action at action object {i} : {expected_response[i][0]}  does not match with the provided action: {actions[i]['action']}"
                                                    return_df.at[index, 'Error Message'] = error_message
                                                    break
                                                else:
                                                    if (int(actions[i]['input']))/60 == expected_response[i][1]:
                                                        error_message = f"Expected wait time: {actions[i]['input']} does not match with the provided wait time: {expected_response[i][1]}"
                                                        return_df.at[index, 'Error Message'] = error_message
                                                        break
                                        else:
                                            error_message = f"Response Error Missing field: 'input' in action object {i}"
                                            return_df.at[index, 'Error Message'] = error_message
                                elif actions[i]['action'] == 'speak':
                                    if 'input' in actions[i]:
                                        if actions[i]['input'] != expected_response[i][1]:
                                            error_message = f"Expected speak message: {expected_response[i]} does not match with the generated speak message: {actions[i]['input']}"
                                            return_df.at[index, 'Error Message'] = error_message
                                            break
                                    else:
                                        error_message = f"Response Error Missing field: 'input' in action object {i}"
                                        return_df.at[index, 'Error Message'] = error_message
                                        break
                else:
                    error_message = "XRIF response is Invalid"
                    return_df.at[index, 'Error Message'] = error_message
                
                pbar.update(1)

        return return_df

In [35]:
xrif = XRIFGenerator(chat_model='deepseek-r1:7b', embed_model = 'deepseek-r1:7b', prompt_template="prompts/xrif_actions_5.yaml", waypoints_csv="waypoint_datasets/warehouse.csv")

Loading model: deepseek-r1:7b...
Processing 24 waypoints...


100%|██████████| 24/24 [00:00<00:00, 18017.41it/s]

Adding waypoints to vector store...


Creating retriever...
Creating prompt template...
Creating RAG chain...


In [ ]:
llm_response, xrif_response = xrif.generate_xrif_with_deepseek_model(query="Go to the North Entrance and then proceed to the C&D - Main Entrance.")

print(xrif_response)
print(llm_response)

In [36]:
test_dataset = pd.read_csv("test_datasets/warehouse_testcases.csv")

test_dataset.columns

Index(['Data Set', 'Experiment ID', 'Experiment Name', 'Prompt ID', 'Prompt',
       'Starting X', 'Starting Y', 'Expected Response',
       'Expected Response Type', 'XRIF Generated', 'Full LLM Response',
       'Notes', 'Error Message'],
      dtype='object')

In [37]:
test_dataset['Expected Response'] = test_dataset['Expected Response'].to_list()

In [38]:
test_dataset.dropna(subset=['Prompt', 'Expected Response'], inplace=True)

In [39]:
result_df = xrif.batch_test_run(test_dataset)

result_df.to_csv("test_results/E7_Test_Results_7b.csv", index=False)

Processing 60 test prompts...


  0%|          | 0/60 [00:00<?, ?it/s]/var/folders/cg/ppl0jhxd3nndl4czw10s0y300000gn/T/ipykernel_59398/3007935440.py:96: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '{'actions': [{'action': 'speak', 'input': 'This waypoint does not exist'}]}' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  return_df.at[index, 'Full LLM Response'] = str(xrif_response)
/var/folders/cg/ppl0jhxd3nndl4czw10s0y300000gn/T/ipykernel_59398/3007935440.py:97: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '{"actions": [{"action": "speak", "input": "This waypoint does not exist"}]}' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  return_df.at[index, 'XRIF Generated'] = json.dumps(xrif_response) if isinstance(xrif_response, dict) else str(xrif_response)
  2%|▏         

KeyboardInterrupt: 

In [17]:
result_df.to_csv("test_results/E7_Test_Results.csv", index=False)